# CMF  
## 分子综合化特征器  
## Comprehensive Molecular Featurizer   
Writed by Yiheng Dai and Yuming Su
### 目标：  
综合多种特征化方式，搭建一体式的化学分子特征化工具  
将容纳多种由后续探索加入的描述符  
### 方法简介：  
内含ECFP(改)方法，版本为6.0
内含RDKit描述符方法
内含自创的共轭结构描述符方法(92个描述符，含5套较复杂的原子级描述符)
内含溶剂与波长合并模块 


In [1]:
# 特征器整体参数：
VERSION = '2.02sym'
import time
# 输入文件：应当包含两列，第一列标题为'smiles'，第二列标题任意，为标签值
INPUT_NAME = 'TPA_856_0307.csv'
LOG_NAME = 'Log_CMF_V'+VERSION+'_'+time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())+'.txt'
SMILES_CHECK = True                       # 进行SMILES检查的开关，开启后若存在错误的smiles，将会终止运行
VALUES_DIV_MOLWT = True                   # 将标签值除以摩尔质量的开关
VALUES_LN = True                          # 将标签值取对数值的开关
PANDAS_DATASET_GENERATE = True            # 生成Pandas可用的数据集的开关

In [2]:
# =====================ECFP part=====================

# ECFP特征器参数：
ECFP_SWITCH = True                        # ECFP特征器的总开关
# 输出文件：一共6个：X，y，特征列标题Title，无标题的SMILES，两个整合的数据集
RADIUS = 4                                # 整数，设置ECFP提取特征的半径

# V1.5加入'特征列填充率保留模块'：
FEATURE_RESERVE = 0.015                  # 浮点数，特征矩阵内非零数据的比例大于等于这个一般保留阈值，这个特征就会被保留
EXACT_PIECE_RESERVE = False                # 布尔值，表示是否开启使用特定基团保留片段的功能
SMILES_PATT = []                      # 字符串列表，每个元素都是一段SMILES值(其实用的是SMART，这样可以匹配类似'ccc'的芳环残片)，用于在保留时作为匹配模式(仅在EXACT_PIECE_RESERVE为True时生效)
EXACT_PIECE_RESERVE_THRESHOLD = 0.015    # 浮点数，针对特定基团的阈值，和上面的类似，若想要全部保留含有某特定基团的特征，此项请用0.0(仅在EXACT_PIECE_RESERVE为True时生效)
# 假如只想保留含有特定基团的片段，可以把FEATURE_RESERVE设为1.0，而EXACT_PIECE_RESERVE设为0.0，因为特定基团筛选的优先级高于一般保留阈值的筛选

# V2.0为针对'CO'的处理而加入，V5.0修改为更具有普遍性的'布尔值转化模块'：
# 作用：将一个max值为n的特征列转化为n或(n+1)列布尔值特征列，分别表示拥有(0),1,2,3,...,n个特征片段
# 注意：若进行全部数据的转化以求获得一个只由{0, 1}构成的特征矩阵，不应和加入RDKit描述符的功能一起使用
# V5.1：添加了自动删除空特征列的功能
# V6.0：整合了两种转化为Bool值的模式，即a版(n列)与b版(n+1列)
TURN_TO_BOOL = False                      # 布尔值，作为布尔值转化模块的总开关使用
ALL_TURN_TO_BOOL = True                   # 布尔值，若为True，则将所有非{0, 1}整数数据全部转化为多个布尔值特征列，最后的特征值矩阵将完全由{0, 1}构成
BOOL_PATT_LIST = []                       # 字符串列表，仅在ALL_TURN_TO_BOOL为False时生效，表示将列表中指定的基团转变为布尔值特征内(注意：必须特征列的标题和这里面的字符串一样才进行转换操作)
BOOL_TURN_MODE = 'a'                      # 字符，'a'或'b'，代表两种布尔值转化的模式，a为n列，b为n+1列

# V3.0加入'RDKit描述符合并模块'，现独立出来作为另一种特征化方法处理
# INCLUDE_DESC = False                      # 布尔值，表示是否需要计算并加入RDKit的描述符

# V4.0加入'片段原子数量调控模块'：
ATOM_COUNT_CONTROL = True                 # 布尔值，作为“原子数量调控模块”的总开关
SMARTS_MIN_LENGTH = 2                     # 整数值，表示特征SMARTS内非氢原子的最小数目，包含非氢原子数目大于等于该数目的基团片段才会被保留，设为0就会跳过这一步骤
SMARTS_MAX_LENGTH = 50                    # 整数值，表示特征SMARTS内非氢原子的最小数目，包含非氢原子数目小于等于该数目的基团片段才会被保留，若不想在最大值上设限，可设为5*RADIUS或更大的数值
ATOM_COUNT_CONTROL_OMIT_PATTERN = []      # 字符串列表，表示若特征片段内含有此列表中的SMARTS片段，此列将不受“原子数量调控模块”的调控

# V4.1加入：新输出一个带Title和Value的完整的数据矩阵

SIMILARITY_SWITCH = False                  # 布尔值，作为“相似度特征化模块”的总开关，使用时请确保TURN_TO_BOOL为False
SIMILARITY_METHOD = 'Tanimoto'            # 字符串，'Tanimoto'或'MACCS'，作为相似度计算的方法
SIMILARITY_MODE = 'mean'                  # 字符串，'max'或'mean'，作为标签值为0的特征的转化方法

In [3]:
# =====================RDKit Descriptor part=====================

# RDKit描述符特征器参数：
# 使用前请先使用Smiles检查程序对数据包中的smiles进行检查，确保
RDKIT_DESC_SWITCH = True           # RDKit描述符特征器的总开关
allowedDescriptors = ['NOCount', 'VSA_EState4', 'NumHDonors', 'SlogP_VSA12', 'NumRadicalElectrons', 
                      'SlogP_VSA4', 'Kappa2', 'Chi2n', 'PEOE_VSA3', 'PEOE_VSA7', 'PEOE_VSA4', 'Chi1', 
                      'MolWt', 'SMR_VSA1', 'SlogP_VSA9', 'VSA_EState9', 'MaxAbsPartialCharge', 'NumSaturatedHeterocycles', 
                      'MaxPartialCharge', 'VSA_EState1', 'PEOE_VSA6', 'EState_VSA11', 'SMR_VSA4', 'EState_VSA7', 
                      'VSA_EState2', 'NHOHCount', 'SlogP_VSA10', 'SMR_VSA7', 'PEOE_VSA9', 'NumAliphaticRings', 'EState_VSA8', 
                      'PEOE_VSA5', 'BertzCT', 'SlogP_VSA1', 'SlogP_VSA6', 'PEOE_VSA1', 'VSA_EState7', 'MinAbsPartialCharge', 
                      'LabuteASA', 'SlogP_VSA2', 'EState_VSA4', 'MolMR', 'Kappa1', 'NumHAcceptors', 'EState_VSA9', 'MolLogP', 
                      'NumAromaticHeterocycles', 'BalabanJ', 'FractionCSP3', 'SMR_VSA3', 'RingCount', 'NumSaturatedRings', 
                      'PEOE_VSA2', 'MaxAbsEStateIndex', 'Kappa3', 'Chi3n', 'NumRotatableBonds', 'Chi4n', 'VSA_EState3', 
                      'SMR_VSA8', 'MinPartialCharge', 'EState_VSA6', 'SMR_VSA9', 'PEOE_VSA13', 'NumValenceElectrons', 
                      'MaxEStateIndex', 'SMR_VSA6', 'VSA_EState8', 'EState_VSA2', 'NumAromaticCarbocycles', 'SMR_VSA10', 
                      'SlogP_VSA3', 'HallKierAlpha', 'PEOE_VSA14', 'HeavyAtomCount', 'VSA_EState10', 'SlogP_VSA11', 
                      'ExactMolWt', 'MinAbsEStateIndex', 'TPSA', 'PEOE_VSA10', 'SMR_VSA2', 'Chi1v', 'Chi4v', 'PEOE_VSA8', 
                      'EState_VSA5', 'Chi1n', 'VSA_EState5', 'SlogP_VSA7', 'HeavyAtomMolWt', 'MinEStateIndex', 
                      'NumAliphaticHeterocycles', 'VSA_EState6', 'Chi0v', 'SlogP_VSA5', 'SMR_VSA5', 'Chi0', 'Chi2v', 
                      'NumSaturatedCarbocycles', 'NumAromaticRings', 'Chi0n', 'PEOE_VSA12', 'Chi3v', 'NumAliphaticCarbocycles', 
                      'EState_VSA10', 'EState_VSA3', 'EState_VSA1', 'NumHeteroatoms', 'SlogP_VSA8', 'PEOE_VSA11']

In [4]:
# =====================Conjugation Descriptor part=====================

# 共轭特征器模块参数：
CONJU_DESC_SWITCH = True                  # 共轭描述符特征器的总开关
KEEP_TYPE = 'max'           # 保留方式，设为'max'、'mean'或'acc-mean'
patt_list_d = ['C=C', 'C#C', 'C#N', 'C=O', 'C=S', 'C=N', 'N=N', '[N+]([O-])=O']
patt_list_m = ['N', 'O', 'S', 'F', 'Cl', 'Br', 'I', 'P']
one_list = ['C']
two_list = ['N', 'O', 'S', 'P', 'F', 'Cl', 'Br', 'I']

In [5]:
# 附加溶剂模块参数：
SOLVENT_SWITCH = True                  # 附加溶剂
SOLVENT_IN = 'Solvent_856.csv'
SOLVENT_TITLE = ['ET(30) (Solvent)', 'Dielectic Constant (Solvent)', 'Dipole Moment (Solvent)']

In [6]:
# 附加波长模块参数：
WAVE_SWITCH = True                  # 附加波长
WAVE_IN = 'Wave_856.csv'
WAVE_TITLE = ['Wavelength (Exp nm)']

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import deepchem as dc
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
from rdkit import DataStructs
from rdkit.Chem import MACCSkeys
from rdkit.Chem.EState.EState import EStateIndices

/home/jyb/.conda/envs/dyhpy/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (4.0.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [8]:
# 创建一个目录来保存生成的数据包
import os
from pathlib import Path
DIR = 'CMF_'+VERSION+'_'+time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())
os.mkdir(DIR)

In [9]:
# 输出数据的创建
X_out = []
title_out = []
smiles_out = []
values_out = []

In [10]:
# 数据集的读取，供DeepChem相关的特征器使用
from deepchem.utils.save import load_from_disk
dataset_file= INPUT_NAME
dataset = load_from_disk(dataset_file)
print("Columns of dataset: %s" % str(dataset.columns.values))
print("Number of examples in dataset: %s" % str(dataset.shape[0]))

deepchem.utils.save has been deprecated.
The utilities in save.py are moved to deepchem.utils.data_utils or deepchem.utils.genomics_utils.


Columns of dataset: ['smiles' 'values']
Number of examples in dataset: 856


In [11]:
# 数据集的读取，供RDKit相关的特征器使用
newdata = []
with open(INPUT_NAME) as file:
    data = file.readlines()
del data[0]
for i in range(len(data)):
    newdata.append(data[i].split(','))
    newdata[i][1] = eval(newdata[i][1])
newdata = np.array(newdata)
smiles_rd = newdata[:, 0].flatten().tolist()
values_rd = newdata[:, 1].astype(float).flatten().tolist()
# 检查Smiles
if SMILES_CHECK:
    error_list = []
    for smi in smiles_rd:
        try:
            mol = Chem.MolFromSmiles(smi)
        except:
            print('error:', smi)
# 将标签值除以分子质量
if VALUES_DIV_MOLWT:
    values_rd_dwt = []
    for descriptor, function in Descriptors.descList:
        if descriptor=='MolWt':
            for i in range(len(smiles_rd)):
                smile = smiles_rd[i]
                mol = Chem.MolFromSmiles(str(smile))
                v = values_rd[i]/function(mol)
                if v==0:
                    print(values_rd[i], function(mol))
                values_rd_dwt.append(v)
    values_dwt_out = np.array(values_rd_dwt).reshape(len(values_rd_dwt), 1)
smiles_out = np.array(smiles_rd).reshape(len(smiles_rd), 1)
values_out = np.array(values_rd).reshape(len(values_rd), 1)

In [12]:
if ECFP_SWITCH:
    # 定义‘特征器’(Featurizer)来将SMILES转化为某些特征
    # featurizer_fix = dc.feat.CircularFingerprint(size=1024)
    featurizer_sparse = dc.feat.CircularFingerprint(size=1024, sparse=True, smiles=True, radius=RADIUS)
    # if INCLUDE_DESC:
        # featurizer_desc = dc.feat.RDKitDescriptors()
    # 对SMILES进行特征化，输出为哈希值以及SMILES块
    # 因为hash值与smiles块一一对应，我们只需挑出unique的SMILES/hash作为特征即可
    loader_sparse = dc.data.CSVLoader(
          tasks=[dataset.columns.values[1]], smiles_field=dataset.columns.values[0],
          featurizer=featurizer_sparse)
    dataset_sparse = loader_sparse.featurize(dataset_file)
    print('Sparse:')
    print(dataset_sparse.get_shape())
    # 此时的特征“矩阵”应当为一个大的字典
    print(dataset_sparse.get_shard(0)[0].shape)
    # 取出所有的分子，这一步得到一个原数据集长度[0]的列表，每个元素为1个字典，字典中为hash:{'smiles':'str', 'count':, int}
    fps_sparse = list(dataset_sparse.get_shard(0)[0])
    smiles = []
    # count = []
    for i in range(dataset_sparse.get_shard(0)[0].shape[0]):
        # 这一步得到的依然是1个字典
        fps_dict = fps_sparse[i]
        for hash_value, sub_dict in fps_dict.items():
            # 这一步把所有的smiles和count挂进列表，count会影响某种特征的权重，也有必要记录下来
            smiles.append(sub_dict['smiles'])
            # count.append(sub_dict['count'])
    print('unique hash:', len(smiles))
    # 这一步得到所有特异的smiles，发现1个smiles可能对应多个hash
    smiles_unique = np.unique(smiles)
    print('unique pieces', len(smiles_unique))
    # 目标是把特征矩阵缩到smiles_unique的长度，这样每一列代表一种特征，特征值即为count的权重
    # 如果这样，可以选择跳过hash，直接统计特征
    title = []
    data = np.zeros((dataset_sparse.get_shard(0)[0].shape[0], len(smiles_unique)-1))
    idx = 0
    index = 0
    for i in range(dataset_sparse.get_shard(0)[0].shape[0]):
        # 这一步得到的依然是1个字典
        fps_dict = fps_sparse[i]
        # 数据提炼去重：
        
        smi = dataset_sparse.get_shard(0)[3][index]
        m = Chem.MolFromSmiles(smi)
            
        for hash_value, sub_dict in fps_dict.items():
            SMILES = sub_dict['smiles']
            patt = Chem.MolFromSmarts(SMILES)
            atomids = m.GetSubstructMatches(patt)
            COUNT = len(atomids)
            # COUNT = sub_dict['count']
            # 跳过'这个空SMILES
            if SMILES=='':
                continue
            if SMILES not in title:
                title.append(SMILES)
                data[i][idx] += COUNT
                idx +=1
            else:
                if data[i][title.index(SMILES)]==0:
                    data[i][title.index(SMILES)] += COUNT
        index += 1
    print(idx)
    print('Before delete sparse coloumns: length of title:', len(title))
    print('Before delete sparse coloumns: shape of data:', data.shape)
    data = data.astype(int)

smiles_field is deprecated and will be removed in a future version of DeepChem.Use feature_field instead.
/home/jyb/.conda/envs/dyhpy/lib/python3.7/site-packages/deepchem/data/data_loader.py:162: FutureWarning: featurize() is deprecated and has been renamed to create_dataset().featurize() will be removed in DeepChem 3.0
  "featurize() will be removed in DeepChem 3.0", FutureWarning)


Sparse:
((856,), (856, 1), (856, 1), (856,))
(856,)
unique hash: 65272
unique pieces 7845
7844
Before delete sparse coloumns: length of title: 7844
Before delete sparse coloumns: shape of data: (856, 7844)


In [13]:
if ECFP_SWITCH:
    # 对于过于稀疏的特征列，设置FEATURE_RESERVE特征保留阈值进行筛选，列填充比例大于此的才会被保留
    # 同时还要保留含有特定基团的片段，此操作的优先级高于按保留阈值筛选
    delete = []
    for i in range(data.shape[1]):
        filled_rate = sum(data[:, i]!=0)/data.shape[0]
        if filled_rate<FEATURE_RESERVE:
            if (EXACT_PIECE_RESERVE):                               # 这一块是在挑出含有指定集团的片段
                m = Chem.MolFromSmarts(title[i])
                for patt_temp in SMILES_PATT:
                    patt = Chem.MolFromSmarts(patt_temp)
                    flag = m.HasSubstructMatch(patt)
                    if flag:
                        break
                if flag and (filled_rate>=EXACT_PIECE_RESERVE_THRESHOLD):
                    continue
            delete.append(i)
    data = np.delete(data, delete, axis=1)
    title = np.array(title).reshape(1, len(title))
    title = np.delete(title, delete, axis=1)
    title = title.flatten().tolist()
    print('After delete sparse coloumns: length of title:', len(title))
    print('After delete sparse coloumns: shape of data:', data.shape)

After delete sparse coloumns: length of title: 564
After delete sparse coloumns: shape of data: (856, 564)


In [14]:
# 原子数量调控模块：
if ECFP_SWITCH and ATOM_COUNT_CONTROL:
    delete = []
    for i in range(data.shape[1]):
        flag = False
        for patt_temp in ATOM_COUNT_CONTROL_OMIT_PATTERN:
            patt = Chem.MolFromSmarts(patt_temp)
            flag = m.HasSubstructMatch(patt)
            if flag:
                break
        if flag:
            continue
        m = Chem.MolFromSmarts(title[i])
        atom_count = len(m.GetAtoms())
        if atom_count<SMARTS_MIN_LENGTH or atom_count>SMARTS_MAX_LENGTH:
            delete.append(i)
    data = np.delete(data, delete, axis=1)
    title = np.array(title).reshape(1, len(title))
    title = np.delete(title, delete, axis=1)
    title = title.flatten().tolist()
    print('After atom-count control: length of title:', len(title))
    print('After atom-count control: shape of data:', data.shape)

After atom-count control: length of title: 564
After atom-count control: shape of data: (856, 564)


In [15]:
# 这一块里，会删除某一指定的特征列，将其转化为表示含有0,1,...,n个基团的布尔表示列，其中n为所有样本中含基团的最大值，最后会自动把空列删除
if ECFP_SWITCH and TURN_TO_BOOL:
    if ALL_TURN_TO_BOOL:
        BOOL_PATT_LIST = title
    for bool_patt in BOOL_PATT_LIST:
        print('Processing feature: ', bool_patt)
        print('title length(before turn to bool):', len(title))
        print('data shape(before turn to bool):', data.shape)
        index_t = title.index(bool_patt)
        print('Target conlumn index:', index_t)
        feature_max = int(max(data[:, index_t]))
        print('max number of target patt:', feature_max)
        # 制作新的bool数据数组
        if BOOL_TURN_MODE=='a':
            matrix_bool = np.zeros((data.shape[0], feature_max))
        else:
            matrix_bool = np.zeros((data.shape[0], feature_max+1))
        for i in range(data.shape[0]):
            if BOOL_TURN_MODE=='a':
                if data[i, index_t]>=1:
                    matrix_bool[i, int(data[i, index_t]-1)] = 1
            else:
                matrix_bool[i, int(data[i, index_t])] = 1
        # 删除对应的数据列和标题列
        data = np.delete(data, [index_t], axis=1)
        title = np.array(title).reshape(1, len(title))
        title = np.delete(title, [index_t], axis=1)
        title = title.flatten().tolist()
        # 制作额外的标题
        new_title = []
        if BOOL_TURN_MODE=='a':
            for i in range(feature_max):
                new_title.append('With_'+str(i+1)+'_'+bool_patt)
        else:
            for i in range(feature_max+1):
                new_title.append('With_'+str(i)+'_'+bool_patt)
        data = np.hstack((data, matrix_bool))
        title = title+new_title
        print('title length(after turn to bool):', len(title))
        print('data shape(after turn to bool):', data.shape, '\n')
    # 将空列自动删除，V5.1加入
    delete = []
    for i in range(data.shape[1]):
        if sum(data[:, i])==0:
            delete.append(i)
    print(data.shape)
    print(len(title))
    data = np.delete(data, delete, axis=1)
    title = np.array(title).reshape(1, len(title))
    title = np.delete(title, delete, axis=1)
    title = title.flatten().tolist()
    print(data.shape)
    print(len(title))

In [16]:
# ECFP配合相似性方法使用
if ECFP_SWITCH and SIMILARITY_SWITCH:
    data = data.astype(float)
    for i in range(data.shape[0]):
        emp = []
        emp_list = []
        fill = []
        fill_list = []
        for j in range(len(title)):
            if data[i, j]>=1:
                fill.append(title[j])
                fill_list.append(j)
            elif data[i, j]==0:
                emp.append(title[j])
                emp_list.append(j)
        for smi1_idx in emp_list:
            similarity = []
            smi1 = title[smi1_idx]
            fps1 = Chem.RDKFingerprint(Chem.MolFromSmarts(smi1))
            for smi2_idx in fill_list:
                smi2 = title[smi2_idx]
                fps2 = Chem.RDKFingerprint(Chem.MolFromSmarts(smi2))
                if SIMILARITY_METHOD=='Tanimoto':
                    sm = DataStructs.FingerprintSimilarity(fps1,fps2)
                elif SIMILARITY_METHOD=='MACCS':
                    sm = DataStructs.FingerprintSimilarity(fps1,fps2,metric=DataStructs.DiceSimilarity)
                sm *= data[i, smi2_idx]
                similarity.append(sm)
            if SIMILARITY_MODE=='max':
                try:
                    sim = max(similarity)
                except:
                    sim = 0.0
            elif SIMILARITY_MODE=='mean':
                try:
                    sim = np.mean(similarity)
                except:
                    sim = 0.0
            data[i, smi1_idx] += sim

In [17]:
# ECFP数据集的整理
if ECFP_SWITCH:
    print(data.shape)
    print(len(title))
    X = data
    y = dataset_sparse.get_shard(0)[1]
    title = np.array(title).reshape(X.shape[1], 1)
    X_out.append(X)
    title_out.append(title)

(856, 564)
564


### =======MFF=======
### =======分割=======
### =======RDKit=======

In [18]:
# RDKit描述符特征器模块
if RDKIT_DESC_SWITCH:
    descriptors = []
    descList = []
    for descriptor, function in Descriptors.descList:
        if descriptor in allowedDescriptors:
            descriptors.append(descriptor)
            descList.append((descriptor, function))
    title_rd = np.array(descriptors).reshape(len(descriptors), 1)
    def _featurize(mol):
        rval = []
        for desc_name, function in descList:
            rval.append(function(mol))
        return rval
    Mol = []
    for i in range(len(smiles_rd)):
        mol = Chem.MolFromSmiles(smiles_rd[i])
        Mol.append(_featurize(mol))
    Mol = np.array(Mol)
    print(Mol.shape)
    X_out.append(Mol)
    title_out.append(title_rd)

(856, 110)


### ======RDKit=======
### ======Conju=======

In [19]:
# 初始化原子半径、键长矩阵，用于计算VSA：
if CONJU_DESC_SWITCH:
    dij_m = np.zeros((54, 54))
    ri_m = np.zeros((54, 1))
    ri_m[6, 0] = 1.950  # C
    ri_m[7, 0] = 1.950  # N
    ri_m[8, 0] = 1.779  # O
    ri_m[9, 0] = 1.496  # F
    ri_m[15, 0] = 2.287  # P
    ri_m[16, 0] = 2.185  # S
    ri_m[17, 0] = 2.044  # Cl
    ri_m[35, 0] = 2.166  # Br
    ri_m[53, 0] = 2.358  # I
    dij_m[6, 35] = 1.970  # C-Br
    dij_m[35, 6] = 1.970
    dij_m[7, 35] = 1.840  # N-Br
    dij_m[35, 7] = 1.840
    dij_m[6, 6] = 1.540  # C-C
    dij_m[7, 7] = 1.450  # N-N
    dij_m[8, 8] = 1.470  # N-N
    dij_m[6, 17] = 1.800  # C-Cl
    dij_m[17, 6] = 1.800
    dij_m[6, 9] = 1.350  # C-F
    dij_m[9, 6] = 1.350
    dij_m[6, 53] = 2.120  # C-I
    dij_m[53, 6] = 2.120
    dij_m[6, 7] = 1.470  # C-N
    dij_m[7, 6] = 1.470
    dij_m[6, 8] = 1.430  # C-N
    dij_m[8, 6] = 1.430
    dij_m[6, 15] = 1.850  # C-P
    dij_m[15, 6] = 1.850
    dij_m[6, 16] = 1.810  # C-S
    dij_m[16, 6] = 1.810
    dij_m[7, 8] = 1.460  # N-O
    dij_m[8, 7] = 1.460
    dij_m[7, 15] = 1.600  # N-P
    dij_m[15, 7] = 1.600
    dij_m[7, 16] = 1.760  # N-S
    dij_m[16, 7] = 1.760
    dij_m[8, 15] = 1.570  # O-P
    dij_m[15, 8] = 1.570
    dij_m[8, 16] = 1.570  # O-S
    dij_m[16, 8] = 1.570

In [20]:
def find_conju(mol, a_m, d_m):
    global patt_list_d, patt_list_m
    ring_list = []
    f_a = []
    patt = Chem.MolFromSmarts('c')
    atomids = mol.GetSubstructMatches(patt)
    atoms = mol.GetAtoms()
    temp_list = []
    
    def find_ring(atom_id, found_atoms):
        nonlocal a_m, ring_list, f_a, atoms, temp_list
        flag = False
        c_list = np.argwhere(a_m[atom_id] == 1).flatten().tolist()
        for atom in c_list:
            if atom not in f_a:
                a = atoms[atom]
                if a.IsInRing() and str(a.GetHybridization())!='SP3':
                    found_atoms.append(atom)
                    f_a.append(atom)
                    find_ring(atom, found_atoms)
                    flag = True
        if not flag:
            temp_list.append(found_atoms)
    
    for atom in atomids:
        a = atom[0]
        if a not in f_a:
            find_ring(a, [])
        if len(temp_list)>0:
            max_ring = temp_list[0]
            for l in temp_list:
                if len(l)>len(max_ring):
                    max_ring = l
            ring_list.append(max_ring)
        temp_list = []
    for patt in patt_list_d:
        f = Chem.MolFromSmarts(patt)
        atomids = mol.GetSubstructMatches(f)
        if len(atomids)>0:
            for pair in atomids:
                n_l = []
                flag_f = False
                for a in pair:
                    if a in f_a:
                        flag_f = True
                        break
                    neighbors = atoms[a].GetNeighbors()
                    for na in neighbors:
                        n_l.append(na.GetIdx())
                if flag_f:
                    continue
                temp = []
                temp_r_id = []
                # 到这里，找到了双、叁键的邻接原子
                for n in n_l:
                    if atoms[n].GetAtomicNum() in [6, 7, 8]:
                        for i in range(len(ring_list)):
                            ring = ring_list[i]
                            if n in ring:
                                temp.append(ring)
                                temp_r_id.append(i)
                if len(temp)==1:
                    ring_list[temp_r_id[0]].append(pair[0])
                    ring_list[temp_r_id[0]].append(pair[1])
                    f_a.append(pair[0])
                    f_a.append(pair[1])
                else:
                    # 合并多个列表
                    t_r = []
                    for r in temp:
                        t_r += r
                    # 加上双、叁键两端
                    t_r.append(pair[0])
                    t_r.append(pair[1])
                    # 删掉原来的环
                    temp_r_id.sort()
                    temp_r_id = np.unique(temp_r_id)
                    for i in reversed(temp_r_id):
                        del ring_list[i]
                    # 加上新环
                    ring_list.append(t_r)
                    f_a.append(pair[0])
                    f_a.append(pair[1])
    for patt in patt_list_m:
        f = Chem.MolFromSmarts(patt)
        atomids = mol.GetSubstructMatches(f)
        if len(atomids)>0:
            for atom in atomids:
                a = atom[0]
                if a not in f_a:
                    neighbors = atoms[a].GetNeighbors()
                    n_l = []
                    for na in neighbors:
                        n_l.append(na.GetIdx())
                    temp = []
                    temp_r_id = []
                    # 到这里，找到了杂原子的邻接原子
                    for n in n_l:
                        for i in range(len(ring_list)):
                            ring = ring_list[i]
                            if (n in ring) and (i not in temp_r_id):
                                temp.append(ring)
                                temp_r_id.append(i)
                    if len(temp)==1:
                        ring_list[temp_r_id[0]].append(a)
                        f_a.append(a)
                    else:
                        # 合并多个列表
                        t_r = []
                        for r in temp:
                            t_r += r
                        # 加上杂原子
                        t_r.append(a)
                        # 删掉原来的环
                        temp_r_id.sort()
                        for i in reversed(temp_r_id):
                            del ring_list[i]
                        # 加上新环
                        if len(t_r)>1:
                            ring_list.append(t_r)
                            f_a.append(a)
    for i in range(len(atoms)):
        if i not in f_a:
            aa = atoms[i]
            if aa.GetSymbol()!='C' or str(aa.GetHybridization())!='SP2':
                continue
            aa_n = aa.GetNeighbors()
            flag = False
            for aaa in aa_n:
                if aaa.GetIdx() in f_a:
                    flag = True
                    break
            if flag:
                a = i
                neighbors = atoms[a].GetNeighbors()
                n_l = []
                for na in neighbors:
                    n_l.append(na.GetIdx())
                temp = []
                temp_r_id = []
                # 到这里，找到了杂原子的邻接原子
                for n in n_l:
                    for i in range(len(ring_list)):
                        ring = ring_list[i]
                        if (n in ring) and (i not in temp_r_id):
                            temp.append(ring)
                            temp_r_id.append(i)
                if len(temp)==1:
                    ring_list[temp_r_id[0]].append(a)
                    f_a.append(a)
                else:
                    # 合并多个列表
                    t_r = []
                    for r in temp:
                        t_r += r
                    # 加上杂原子
                    t_r.append(a)
                    # 删掉原来的环
                    temp_r_id.sort()
                    for i in reversed(temp_r_id):
                        del ring_list[i]
                    # 加上新环
                    if len(t_r)>1:
                        ring_list.append(t_r)
                        f_a.append(a)
    # 最后核对共轭结构是否相连
    if len(ring_list)>1:
        temp_count = 0
        flag = True
        while flag:
            t_temp = int(len(ring_list)*(len(ring_list)-1)/2)
            temp = 0
            break_flag = False
            for i in range(len(ring_list)-1):
                for j in range(len(ring_list)-i-1):
                    r_1 = ring_list[i]
                    r_2 = ring_list[i+j+1]
                    if np.sum(a_m[r_1, :][:, r_2]) == 0:
                        temp += 1
                    else:
                        # 需要进行合并
                        for k in r_2:
                            ring_list[i].append(k)
                        ring_list[i] = np.unique(ring_list[i])
                        del ring_list[i+j+1]
                        break_flag = True
                        break
                if break_flag:
                    break
            if temp == t_temp:
                flag = False
    for i in range(len(ring_list)):
        ring_list[i] = np.unique(ring_list[i]).flatten().tolist()
    return (ring_list, f_a)

In [21]:
def find_elec_num(kind, hyb):
    global one_list, two_list
    if kind in one_list:
        return 1
    elif kind in two_list:
        if hyb=='SP' and kind in ['N', 'P', 'O']:
            return 1
        elif hyb=='SP2' and kind in ['N', 'O', 'S']:
            return 1
        else:
            return 2

In [22]:
def find_acc_mean(l_in, conju_size_list):
    temp = 0
    for i in range(len(l_in)):
        temp += l_in[i] * conju_size_list[i]
    temp /= sum(conju_size_list)
    return temp

In [31]:
# 共轭描述符特征器模块
if CONJU_DESC_SWITCH:
    CONJU_TITLE = []
    CONJU_PRE = ['Apperant-Elec-Count', 'PEOE-Charge', 'EState-Indice', 'Atomic-LogP', 'Atomic-MR']
    CONJU_NUM = 5 + 7 + 16 * len(CONJU_PRE)
    data_conju = np.zeros((len(smiles_rd), CONJU_NUM))
    descList = []
    allowedDescriptors = ['MolWt']
    for descriptor, function in Descriptors.descList:
        if descriptor in allowedDescriptors:
            descList.append((descriptor, function))
    mff_title = title_out[0].flatten().tolist()
    for _ in range(len(smiles_rd)):
        smi = smiles_rd[_]
        mol = Chem.MolFromSmiles(Chem.MolToSmiles(Chem.MolFromSmiles(smi)))
        atoms = mol.GetAtoms()
        a_m = Chem.rdmolops.GetAdjacencyMatrix(mol)
        d_m = Chem.rdmolops.GetDistanceMatrix(mol)
        res = find_conju(mol, a_m, d_m)
        ring_list = res[0]
        lstmax = sorted(ring_list, key = len)[-1]
        f_a = res[1]
        conju_size_list = [len(r) for r in ring_list]
        ## 计算原子级描述符列表：
        # 计算表观共轭电荷
        app_elec = []
        for a in range(len(atoms)):
            if a in f_a:
                a_kind = atoms[a].GetSymbol()
                hyb = str(atoms[a].GetHybridization())
                app_elec.append(find_elec_num(a_kind, hyb))
            else:
                app_elec.append(0)
        # 计算PEOE电荷
        AllChem.ComputeGasteigerCharges(mol, nIter=25)
        peoe_charge = [mol.GetAtomWithIdx(i).GetDoubleProp('_GasteigerCharge') for i in range(mol.GetNumAtoms())]
        # 计算电子拓扑系数EState
        estate_index = EStateIndices(mol)
        # 计算LogP与MR的原子贡献
        contribs = rdMolDescriptors._CalcCrippenContribs(mol)
        logp = [contribs[i][0] for i in range(len(contribs))]
        mr = [contribs[i][1] for i in range(len(contribs))]
        # 结合各原子级描述符
        atom_props = [app_elec, peoe_charge, estate_index, logp, mr]
        
        ## 共轭结构的总和特征：
        if _ == 0:
            CONJU_TITLE.append('Num-Conju-Stru')  # 1.
            CONJU_TITLE.append('Conju-Num-Atom-All')  # 2.
            CONJU_TITLE.append('Conju-Num-Atom-Ratio')  # 3.
            CONJU_TITLE.append('Conju-Wt-Ave')  # 4.
            CONJU_TITLE.append('Full-Mol Wiener Index')  # 5.
            CONJU_TITLE.append('Conju-Num-Atom-Individual')  # 6.
            CONJU_TITLE.append('Conju-Wt-Part')  # 7.
            CONJU_TITLE.append('Conju-Wt-Ave')  # 8.
            CONJU_TITLE.append('Conju-Max-Distance')  # 9.
            CONJU_TITLE.append('Conju-Branch-Ratio')  # 10.
            CONJU_TITLE.append('Conju-Stru-Wiener-Index')  # 11.
            CONJU_TITLE.append('Conju-Stru-VSA')  # 12.
        # 1.共轭结构数量
        data_conju[_, 0] = len(ring_list)
        # 2.共轭原子总数
        data_conju[_, 1] = len(f_a)
        # 3.共轭结构数量占比
        data_conju[_, 2] = len(f_a)/len(atoms)
        # 4.共轭结构质量占比
        rval = []
        for desc_name, function in descList:
            rval.append(function(mol))
        wt_list = []
        mwt_list = []
        for r in ring_list:
            tt = 0
            for a in r:
                tt += atoms[a].GetMass()
            wt_list.append(tt)
            mwt_list.append(tt/len(r))
        data_conju[_, 3] = sum(wt_list)/rval[0]
        # 5.全分子维纳指数
        if '.' in smi:
            mol22 = Chem.MolFromSmiles(smi.split('.')[0])
            dm22 = Chem.rdmolops.GetDistanceMatrix(mol22)
            data_conju[_, 4] = np.sum(dm22)/(2*dm22.shape[0]*(dm22.shape[0]-1))
        else:
            data_conju[_, 4] = np.sum(d_m)/(2*d_m.shape[0]*(d_m.shape[0]-1))
        
        ## 共轭结构独立特征
        conju_props = []
        # 6.共轭结构独立原子数
        size_l = []
        for r in ring_list:
            size_l.append(len(r))
        conju_props.append(size_l)
        
        # 7.共轭结构独立质量
        # 8.共轭结构独立原子平均质量
        conju_props.append(wt_list)
        conju_props.append(mwt_list)
        # 9. 共轭结构长度
        conju_max_dis = []
        for r in ring_list:
            conju_max_dis.append(np.max(d_m[r, :][:, r]))
        conju_props.append(conju_max_dis)
        
        # 10.共轭结构分支系数
#         branch_l = []
#         for i in range(len(wt_list)):
#             branch_l.append(np.sum(a_m[ring_list[i], :][:, ring_list[i]])/(2*size_l[i]))
    
        for i in range(len(ring_list)):
            d_mring=d_m[ring_list[i], :][:, ring_list[i]] 
        dlist=np.where(d_mring==np.max(d_mring))
        l_dlist=len(dlist[0])/2
        ltemp=dlist[0][0:round(l_dlist)]
        distemp=np.max(ltemp)-np.min(ltemp)
        conjuratio=[distemp/max(conju_max_dis)]        
        conju_props.append(conjuratio)
        # 11.共轭结构维纳指数
        wi_l = []
        for r in ring_list:
            d_m_temp = d_m[r, :][:, r]
            wi_l.append(np.sum(d_m_temp)/(2*d_m_temp.shape[0]*(d_m_temp.shape[0]-1)))
        conju_props.append(wi_l)
        
        # 12.共轭结构VSA
        conju_vsa_l = []
        for r in ring_list:
            vsa_t = 0
            for i in range(len(r)):
                vsa_tt = 0
                atom = atoms[r[i]]
                n_l = atom.GetNeighbors()
                aid_1 = atom.GetAtomicNum()
                ar_1 = ri_m[aid_1, 0]
                for j in range(len(n_l)):
                    aid_2 = n_l[j].GetAtomicNum()
                    ar_2 = ri_m[aid_2, 0]
                    dij_i = dij_m[aid_1, aid_2]
                    dij = min(max(abs(ar_1-ar_2), dij_i), ar_1+ar_2)
                    vsa_tt += (ar_2**2-(ar_1-dij)**2)/dij
                vsa_t += 4*np.pi*ar_1**2 - np.pi*ar_1*vsa_tt
            conju_vsa_l.append(vsa_t)
        conju_props.append(conju_vsa_l)
        
        # 接下来计算原子描述符的性质
        for __ in range(len(atom_props)):
            PRE = 'Conju-'+CONJU_PRE[__]+'-'
            PRE = CONJU_PRE[__]+'-'
            END_P = ' (MFF-Conju)'
            atom_props_list = atom_props[__]
            if _ == 0:
                CONJU_TITLE.append(PRE+'Sum')  # 13.1.
                CONJU_TITLE.append(PRE+'Ave')  # 13.2.
                CONJU_TITLE.append(PRE+'Max')  # 13.3.
                CONJU_TITLE.append(PRE+'Min')  # 13.4.
                CONJU_TITLE.append(PRE+'Delta')  # 13.5. 
                CONJU_TITLE.append(PRE+'Weighted')  # 13.6.
                CONJU_TITLE.append(PRE+'Weighted-Ave')  # 13.7.
                CONJU_TITLE.append(PRE+'PositiveDisCoef')  # 13.8.
                CONJU_TITLE.append(PRE+'PositiveDisCoef-PairMean')  # 13.9.
                CONJU_TITLE.append(PRE+'NegativeDisCoef')  # 13.10.
                CONJU_TITLE.append(PRE+'NegativeDisCoef-PairMean')  # 13.11.
                CONJU_TITLE.append(PRE+'GradSum')  # 13.12.
                CONJU_TITLE.append(PRE+'GradSum-PairMean')  # 13.13. 
                CONJU_TITLE.append(PRE+'MaxMinDisRatio')  # 13.14.
                # CONJU_TITLE.append(PRE+'AppearantU')  # 13.15.
                CONJU_TITLE.append(PRE+'LaplaceSum')  # 13.16.
                CONJU_TITLE.append(PRE+'Laplace-PairMean')  # 13.17.
            # 13.1. 不含碎片的求和
            x_count_l = []
            # 13.2. 不含碎片的按原子平均的平均值
            x_atom_mean_l = []
            # 13.3. 含碎片的最大值
            x_max_l = []
            # 13.4. 含碎片的最小值
            x_min_l = []
            # 13.5. 含碎片的差值
            x_delta_l = []
            # 13.6. 影响力
            x_infl_l = []
            # 13.7. 按原子平均的影响力
            x_atom_infl_l = []
            # 13.8. 一阶距离影响力系数
            x_pos_dis_coef_l = []
            # 13.9. 按二元原子对数目平均的一阶距离影响力系数
            x_pair_mean_pos_dis_coef_l = []
            # 13.10. 负一阶距离影响力系数
            x_neg_dis_coef_l = []
            # 13.11. 按二元原子对数目平均的负一阶距离影响力系数
            x_pair_mean_neg_dis_coef_l = []
            # 13.12. 总一阶梯度
            x_sum_grad_l = []
            # 13.13. 按二元原子对数目平均的一阶梯度
            x_pair_mean_grad_l = []
            # 13.14. 含碎片的极值-极值距离占比
            x_dis_ratio_l = []
            # 13.15. 按电荷性质与图论距离计算的势能（与负一阶一样，删除掉）
            # x_u_l = []
            # 13.16. 总二阶梯度
            x_sum_laplace_l = []
            # 13.17. 按二元原子对数目平均的二阶梯度
            x_pair_mean_laplace_l = []
            # 开始计算性质
            for r in [lstmax]:
                # 整合1.2.
                a_p_l = [atom_props_list[a] for a in r]
                x_count_l.append(sum(a_p_l))  # 1.
                x_atom_mean_l.append(sum(a_p_l)/len(r))  # 2.
                # 生成碎片性质列表
                frag_x = []
                frag_atom_id = []
                for i in range(len(mff_title)):
                    patt = mff_title[i]
                    f = Chem.MolFromSmarts(patt)
                    atomids = mol.GetSubstructMatches(f)
                    if len(atomids) > 0:
                        for j in range(len(atomids)):
                            peoe_flag = True
                            for k in atomids[j]:
                                if k not in r:
                                    peoe_flag = False
                                    break
                            if peoe_flag:
                                frag_atom_id.append(atomids[j])
                                x_temp = 0
                                for k in atomids[j]:
                                    x_temp += atom_props_list[k]
                                frag_x.append(x_temp)
#                     for j in range(len(r)):
#                         atom_id = r[j]
#                         frag_x.append(atom_props_list[atom_id])
#                         frag_atom_id.append([atom_id])
                if len(frag_x)==0:
                    frag_x=[0]
                    frag_atom_id=[()]
                x_max_l.append(max(frag_x))  # 3.
                x_min_l.append(min(frag_x))  # 4.
                x_delta_l.append(max(frag_x) - min(frag_x))  # 5.
                f_1 = [i for i in frag_atom_id[frag_x.index(min(frag_x))]]
                f_2 = [i for i in frag_atom_id[frag_x.index(max(frag_x))]]
                s = np.max(d_m[f_1, :][:, f_2])/np.max(d_m[r, :][:, r])
                x_dis_ratio_l.append(s)  # 14.
                # 计算影响力
                temp_infl = 0
                for a in r:
                    temp_infl += atom_props_list[a] * atoms[a].GetDegree()
                x_infl_l.append(temp_infl)  # 6.
                x_atom_infl_l.append(temp_infl/len(r))  # 7.
                # 计算距离系数与一阶、二阶梯度
                ttp = 0
                tti = 0
                grad_temp = 0
                laplace_temp = 0
                count = 0
                for i in range(len(r)):
                    for j in range(len(r)-i-1):
                        a = r[i]
                        b = r[i+j+1]
                        count += 1
                        a_e = atom_props_list[a]
                        b_e = atom_props_list[b]
                        ttp += a_e*b_e*d_m[a, b]
                        tti += a_e*b_e/d_m[a, b]
                        grad_temp += abs(a_e-b_e)/d_m[a, b]
                        laplace_temp += abs(a_e-b_e)/(d_m[a, b]**2)
                x_pos_dis_coef_l.append(ttp)  # 8.
                x_pair_mean_pos_dis_coef_l.append(ttp/count)  # 9.
                x_neg_dis_coef_l.append(tti)  # 10.
                x_pair_mean_neg_dis_coef_l.append(tti/count)  # 11.
                x_sum_grad_l.append(grad_temp)  # 12.
                x_pair_mean_grad_l.append(grad_temp/count)  # 13.
                x_sum_laplace_l.append(laplace_temp)  # 16.
                x_pair_mean_laplace_l.append(laplace_temp/count)  # 17.
            conju_props.append(x_count_l)
            conju_props.append(x_atom_mean_l)
            conju_props.append(x_max_l)
            conju_props.append(x_min_l)
            conju_props.append(x_delta_l)
            conju_props.append(x_infl_l)
            conju_props.append(x_atom_infl_l)
            conju_props.append(x_pos_dis_coef_l)
            conju_props.append(x_pair_mean_pos_dis_coef_l)
            conju_props.append(x_neg_dis_coef_l)
            conju_props.append(x_pair_mean_neg_dis_coef_l)
            conju_props.append(x_sum_grad_l)
            conju_props.append(x_pair_mean_grad_l)
            conju_props.append(x_dis_ratio_l)
            # conju_props.append(x_u_l)
            conju_props.append(x_sum_laplace_l)
            conju_props.append(x_pair_mean_laplace_l)
        # 开始填入特征
        for i in range(len(conju_props)):
            index = 5 + i
            if KEEP_TYPE == 'max':
                data_conju[_, index] = max(conju_props[i])
            elif KEEP_TYPE == 'mean':
                data_conju[_, index] = mean(conju_props[i])
            elif KEEP_TYPE == 'acc-mean':
                temp = 0
                for j in range(len(conju_size_list)):
                    temp += conju_size_list[j] * conju_props[i][j]
                data_conju[_, index] = temp / sum(conju_size_list)
    X_out.append(data_conju)
    title_out.append(np.array(CONJU_TITLE).reshape(len(CONJU_TITLE), 1))

In [36]:
if SOLVENT_SWITCH:
    sol_d = np.loadtxt(SOLVENT_IN, dtype=float, delimiter=',')
    X_out.append(sol_d)
    title_out.append(np.array(SOLVENT_TITLE).reshape(3, 1))

In [37]:
if WAVE_SWITCH:
    wave_d = np.loadtxt(WAVE_IN, dtype=float, delimiter=',')
    data_w = np.zeros((len(smiles_rd), 1))
    for i in range(len(smiles_rd)):
        data_w[i, 0] = wave_d[i, ]
    X_out.append(data_w)
    title_out.append(np.array(WAVE_TITLE).reshape(1, 1))

In [ ]:
# 预留给更多的特征化模块的空间

In [38]:
# 将所有对应的数据包叠加起来，得到一个大数据包
X_init = X_out[0]
if len(X_out)>=2:
    for i in range(len(X_out)-1):
        X_init = np.hstack((X_init, X_out[i+1]))
title_init = title_out[0]
if len(title_out)>=2:
    for i in range(len(title_out)-1):
        title_init = np.vstack((title_init, title_out[i+1]))
title_init = np.transpose(title_init)
print(X_init.shape, title_init.shape)
title_extra = ['smiles', 'values']
data_out = np.hstack((X_init, smiles_out))
data_out = np.hstack((data_out, values_out))
if VALUES_DIV_MOLWT:
    title_extra.append('values_dwt')
    data_out = np.hstack((data_out, values_dwt_out))
if VALUES_LN:
    title_extra.append('values_ln')
    data_out = np.hstack((data_out, np.log(values_out)))
    if VALUES_DIV_MOLWT:
        title_extra.append('values_dwt_ln')
        data_out = np.hstack((data_out, np.log(values_dwt_out)))
title_extra = np.array(title_extra).reshape(1, len(title_extra))
title_out = np.hstack((title_init, title_extra))
full_data = np.vstack((title_out, data_out))
print(full_data.shape)

(856, 770) (1, 770)
(857, 775)


In [39]:
# 删除含有nan或infi的行，以及只有0的列
del_list = []
for i in range(full_data.shape[0]-1):
    j = i+1
    if (np.isnan(full_data[j, :-5].astype(float)).sum()>0)or(np.inf in full_data[j, :-5].astype(float)):
        del_list.append(j)
        continue
print(full_data[0, del_list])
full_data = np.delete(full_data, del_list, axis=0)
if X_init.shape[0]>1:
    del_list = []
    for i in range(X_init.shape[1]):
        if max(full_data[1:, i].astype(float))==min(full_data[1:, i].astype(float)):
            del_list.append(i)
            continue
    print(full_data[0, del_list])
    full_data = np.delete(full_data, del_list, axis=1)
print(full_data.shape)

[]
['NumRadicalElectrons' 'SMR_VSA8' 'SlogP_VSA9']
(857, 772)


In [40]:
# 将大数据包拆分开来
OUT_NAME_FULL = 'Full_'+str(full_data.shape[0])+'_'+str(full_data.shape[1])+'.csv'
OUT_NAME_FULL = Path('.', DIR, OUT_NAME_FULL)
np.savetxt(OUT_NAME_FULL, full_data, fmt='%s', delimiter=',')
data_t = full_data[1:, :]
full_t = full_data
if VALUES_LN:
    if VALUES_DIV_MOLWT:
        values_out_wt_ln = data_t[:, -1]
        OUT_NAME_VALUES_WT_LN = 'Values_True_w_ln_'+str(values_out_wt_ln.shape[0])+'.csv'
        OUT_NAME_VALUES_WT_LN = Path('.', DIR, OUT_NAME_VALUES_WT_LN)
        np.savetxt(OUT_NAME_VALUES_WT_LN, values_out_wt_ln, fmt='%s', delimiter=',')
        data_t = data_t[:, :-1]
        full_t = full_t[:, :-1]
    values_out_ln = data_t[:, -1]
    OUT_NAME_VALUES_LN = 'Values_True_ln_'+str(values_out_ln.shape[0])+'.csv'
    OUT_NAME_VALUES_LN = Path('.', DIR, OUT_NAME_VALUES_LN)
    np.savetxt(OUT_NAME_VALUES_LN, values_out_ln, fmt='%s', delimiter=',')
    data_t = data_t[:, :-1]
    full_t = full_t[:, :-1]
if VALUES_DIV_MOLWT:
    values_out_wt = data_t[:, -1]
    OUT_NAME_VALUES_WT = 'Values_True_w_'+str(values_out_wt.shape[0])+'.csv'
    OUT_NAME_VALUES_WT = Path('.', DIR, OUT_NAME_VALUES_WT)
    np.savetxt(OUT_NAME_VALUES_WT, values_out_wt, fmt='%s', delimiter=',')
    data_t = data_t[:, :-1]
    full_t = full_t[:, :-1]
values_out = data_t[:, -1]
OUT_NAME_VALUES = 'Values_True_'+str(values_out.shape[0])+'.csv'
OUT_NAME_VALUES = Path('.', DIR, OUT_NAME_VALUES)
np.savetxt(OUT_NAME_VALUES, values_out, fmt='%s', delimiter=',')
data_t = data_t[:, :-1]
full_t = full_t[:, :-1]
smiles_out = data_t[:, -1]
OUT_NAME_SMILES = 'Smiles_'+str(values_out.shape[0])+'.csv'
OUT_NAME_SMILES = Path('.', DIR, OUT_NAME_SMILES)
np.savetxt(OUT_NAME_SMILES, smiles_out, fmt='%s', delimiter=',')
X_out = data_t[:, :-1]
full_t = full_t[:, :-1]
OUT_NAME_X = 'Features_'+str(X_out.shape[0])+'_'+str(X_out.shape[1])+'.csv'
OUT_NAME_X = Path('.', DIR, OUT_NAME_X)
np.savetxt(OUT_NAME_X, X_out, fmt='%s', delimiter=',')
OUT_NAME_TITLE = 'Title_'+str(full_t.shape[1])+'.csv'
OUT_NAME_TITLE = Path('.', DIR, OUT_NAME_TITLE)
np.savetxt(OUT_NAME_TITLE, np.transpose(full_t[0, :]), fmt='%s', delimiter=',')

In [ ]:
# 输出综合的数据集，供pandas使用
if PANDAS_DATASET_GENERATE:
    dataset_out = np.hstack((full_t, np.hstack((np.array(['values']), values_out)).reshape(full_t.shape[0], 1)))
    OUT_NAME_D1 = 'Dataset_'+str(X_out.shape[0])+'_'+str(X_out.shape[1])+'.csv'
    OUT_NAME_D1 = Path('.', DIR, OUT_NAME_D1)
    np.savetxt(OUT_NAME_D1, dataset_out, fmt='%s', delimiter=',')
    if VALUES_DIV_MOLWT:
        dataset_out = np.hstack((full_t, np.hstack((np.array(['values_wt']), values_out_wt)).reshape(full_t.shape[0], 1)))
        OUT_NAME_D2 = 'Dataset_w_'+str(X_out.shape[0])+'_'+str(X_out.shape[1])+'.csv'
        OUT_NAME_D2 = Path('.', DIR, OUT_NAME_D2)
        np.savetxt(OUT_NAME_D2, dataset_out, fmt='%s', delimiter=',')
    if VALUES_LN:
        dataset_out = np.hstack((full_t, np.hstack((np.array(['values_ln']), values_out_ln)).reshape(full_t.shape[0], 1)))
        OUT_NAME_D3 = 'Dataset_ln_'+str(X_out.shape[0])+'_'+str(X_out.shape[1])+'.csv'
        OUT_NAME_D3 = Path('.', DIR, OUT_NAME_D3)
        np.savetxt(OUT_NAME_D3, dataset_out, fmt='%s', delimiter=',')
        if VALUES_DIV_MOLWT:
            dataset_out = np.hstack((full_t, np.hstack((np.array(['values_wt_ln']), values_out_wt_ln)).reshape(full_t.shape[0], 1)))
            OUT_NAME_D4 = 'Dataset_w_ln_'+str(X_out.shape[0])+'_'+str(X_out.shape[1])+'.csv'
            OUT_NAME_D4 = Path('.', DIR, OUT_NAME_D4)
            np.savetxt(OUT_NAME_D4, dataset_out, fmt='%s', delimiter=',')

In [ ]:
# 输出Log文件
LOG_NAME = Path('.', DIR, LOG_NAME)
f1 = open(LOG_NAME, 'w+')
f1.write('Log for CMF\n')
f1.write('Version: V'+VERSION+'\n\n')
f1.write('Log generation time: '+time.strftime("%Y.%m.%d-%H:%M:%S", time.localtime())+'\n\n')
if SMILES_CHECK:
    f1.write('Smiles check is on.\n')
if VALUES_DIV_MOLWT:
    f1.write('Data with values divided by molecular weight is generated.\n')
if VALUES_LN:
    f1.write('Data with ln(values) is generated.\n')
if PANDAS_DATASET_GENERATE:
    f1.write('Dataset(s) which can be used by pandas is(are) generated.\n')
f1.write('\n\n')
f1.write('Parameters:\n\n')
if ECFP_SWITCH:
    f1.write('ECFP(modified) featurizer is on.\n')
    f1.write('ECFP radius: '+str(RADIUS)+'\n')
    f1.write('Rate of feature reservation: '+str(FEATURE_RESERVE)+'\n')
    if EXACT_PIECE_RESERVE:
        f1.write('   Exact smiles piece reservation is on.\n')
        f1.write('   Smiles piece(s): '+str(SMILES_PATT)+'\n')
        f1.write('   Rate of feature reservation for exact smiles piece(s): '+str(EXACT_PIECE_RESERVE_THRESHOLD)+'\n')
    if TURN_TO_BOOL:
        f1.write('Turn-to-bool module is on.\n')
        f1.write('Turn-to-bool mode: '+BOOL_TURN_MODE+'\n')
        if ALL_TURN_TO_BOOL:
            f1.write('All features will be turned to bool values.\n')
        else:
            if len(BOOL_PATT_LIST)>0:
                f1.write('Smiles pieces to be turned into bool: '+str(BOOL_PATT_LIST)+'\n')
    if ATOM_COUNT_CONTROL:
        f1.write('Atom-count-control module is on.\n')
        f1.write('Minimum atom (except H) number of allowed smarts piece: '+str(SMARTS_MIN_LENGTH)+'\n')
        f1.write('Maximum atom (except H) number of allowed smarts piece: '+str(SMARTS_MAX_LENGTH)+'\n')
        if len(ATOM_COUNT_CONTROL_OMIT_PATTERN)>0:
            f1.write('Excepption smarts pieces of Atom-count-control module: '+str(ATOM_COUNT_CONTROL_OMIT_PATTERN)+'\n')
    if SIMILARITY_SWITCH:
        f1.write('Fingerprint-Similarity module is on.\n')
        f1.write('Fingerprint-Similarity method: '+SIMILARITY_METHOD+'\n')
        f1.write('Fingerprint-Similarity mode: '+SIMILARITY_MODE+'\n')
    f1.write('\n')
if RDKIT_DESC_SWITCH:
    f1.write('RDKit Descriptor featurizer is on.\n')
    f1.write('Length of allowed descriptor set: '+str(len(allowedDescriptors))+'\n')
    f1.write('Allowed descriptors:\n')
    for i in range(len(allowedDescriptors)):
        if (i)%5 == 0:
            f1.write('    ')
        f1.write(allowedDescriptors[i]+'   ')
        if (i+1)%5 == 0:
            f1.write('\n')
    f1.write('\n')
f1.write('Some samples may be deleted beacause of the presence of nan or infinite.\n')
f1.write('Some columns may be deleted beacause of only 0 is contained.\n')
f1.close()